In [26]:
#from sklearn.cluster import KMeans
import numpy as np
import scipy as sp
import sys
sys.path.insert(0, '../corv/src')
sys.path.append('../wdtools')

import matplotlib.pyplot as plt
import matplotlib.colors as colors
from astropy.io import fits
from astropy.table import Table
from tqdm import tqdm

### Query
from astroquery.sdss import SDSS
from astroquery.gaia import Gaia
from astropy import constants as c

#import wdtools
import corv


In [27]:
from astropy.table import Table, vstack, unique

ms_sdss4_wd_sdss4 = Table.read('data/ms_sdss4_wd_sdss4.fits')
ms_gaia_wd_sdss4 = Table.read('data/ms_gaia_wd_sdss4.fits')
ms_sdss5_wd_sdss4 = Table.read('sdss5/ms_sdss5_wd_sdss4.fits')

ms_gaia_wd_sdss5 = Table.read('sdss5/ms_gaia_wd_sdss5.fits')
ms_sdss4_wd_sdss5 = Table.read('sdss5/ms_sdss4_wd_sdss5.fits')
ms_sdss5_wd_sdss5 = Table.read('sdss5/ms_sdss5_wd_sdss5.fits')

def strip(catalog):  
    drop_cols = []
    keep_cols = ['wd_source_id', 'wd_ra', 'wd_dec', 'wd_plate', 'wd_mjd', 'wd_fiberid', 'wd_filepath', 'wd_rv_from']
        
    for key in catalog.keys():
        if key not in keep_cols:
            drop_cols.append(key)
            
    catalog.remove_columns(drop_cols)
    
    return catalog

ms_gaia_wd_sdss4  = strip(ms_gaia_wd_sdss4 )
ms_sdss4_wd_sdss4 = strip(ms_sdss4_wd_sdss4)
ms_sdss5_wd_sdss4 = strip(ms_sdss5_wd_sdss4)

ms_gaia_wd_sdss5  = strip(ms_gaia_wd_sdss5 )
ms_sdss4_wd_sdss5 = strip(ms_sdss4_wd_sdss5)
ms_sdss5_wd_sdss5 = strip(ms_sdss5_wd_sdss5)

catalog = unique(vstack([ms_sdss4_wd_sdss4, ms_gaia_wd_sdss4, ms_sdss4_wd_sdss5, ms_sdss5_wd_sdss4, ms_gaia_wd_sdss5, ms_sdss5_wd_sdss5]), keys = 'wd_source_id')

In [28]:
catalog.keys()

['wd_source_id',
 'wd_ra',
 'wd_dec',
 'wd_plate',
 'wd_mjd',
 'wd_fiberid',
 'wd_rv_from',
 'wd_filepath']

In [29]:
catalog['wd_filepath'][-1].split('-')

['spec', '101076', '59872', '27021597926091959.fits']

In [30]:
paths = []
internetpaths = []

for i in tqdm(range(len(catalog))):
    if catalog['wd_rv_from'][i] == 'sdss5':
        catalog['wd_filepath'][i] = catalog['wd_filepath'][i].replace('spec-','spSpec-')
        paths.append('sdss5/wd_fits/' + catalog['wd_filepath'][i])
        a, b = catalog['wd_filepath'][i].split('-')[1], catalog['wd_filepath'][i].split('-')[2]
        
        internetpaths.append('https://data.sdss5.org/sas/sdsswork/bhm/boss/spectro/redux/v6_1_0/' + a + '/coadd/' + b + '/' + catalog['wd_filepath'][i])
    else:
        paths.append('')
        
catalog['paths'] = paths

100%|██████████| 409/409 [00:00<00:00, 18666.91it/s]


In [31]:
i = 0

spec = SDSS.get_spectra(plate=catalog['wd_plate'][i], fiberID=catalog['wd_fiberid'][i], mjd=catalog['wd_mjd'][i])[0]
spec

/home/sarsene1/.local/lib/python3.8/site-packages/astroquery/sdss/core.py:874: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  arr = np.atleast_1d(np.genfromtxt(io.BytesIO(response.content),


[<astropy.io.fits.hdu.image.PrimaryHDU object at 0x1554f1f20c40>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x1554fa76e7f0>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x155506bca880>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x155506b95490>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x155506c36b20>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x1554fa3dc430>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x1554fa1182b0>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x155506bbc4c0>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x155506d56d60>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x155504012550>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x15552a1cd280>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x155506e35130>]

In [32]:
spec = []
wl = []
fl = []
ivar = []
drops = []
worked = []

for i in tqdm(range(len(catalog))):
    if catalog['paths'][i] == '':
        try:
            spec = SDSS.get_spectra(plate=catalog['wd_plate'][i], fiberID=catalog['wd_fiberid'][i], mjd=catalog['wd_mjd'][i])[0]
            
            #print(spec[4].header)
            
            wl.append(10**spec[1].data['loglam'])
            fl.append(spec[1].data['flux'])
            ivar.append(spec[1].data['ivar'])
            worked.append(i)
        except:
            drops.append(i)
    else:
        spec = fits.open(catalog['paths'][i])
        wl.append(10**spec[1].data['loglam'])
        fl.append(spec[1].data['flux'])
        ivar.append(spec[1].data['ivar'])
        worked.append(i)
        
print(len(wl))
print(len(fl))
print(len(ivar))
        
catalog.remove_rows(drops)
catalog['wd_wl'] = wl
catalog['wd_fl'] = fl
catalog['wd_ivar'] = ivar

snr = []

for i in range(len(catalog)):
    wavl_range = (5000<catalog['wd_wl'][i])*(catalog['wd_wl'][i]<6000)
    snr.append(np.median(catalog['wd_fl'][i][wavl_range] * np.sqrt(catalog['wd_ivar'][i][wavl_range])))
    
catalog['snr'] = snr

100%|██████████| 409/409 [00:03<00:00, 134.55it/s]

409
409
409


In [33]:
wavl_range

array([False, False, False, ..., False, False, False])

In [34]:
catalog

wd_source_id,wd_ra,wd_dec,wd_plate,wd_mjd,wd_fiberid,wd_rv_from,wd_filepath,paths,wd_wl,wd_fl,wd_ivar,snr
int64,float64,float64,int64,int64,int64,bytes5,bytes42,str56,object,object,object,float32
78630783787018624,29.9587008420414,15.831669889570092,5119,55836,240,sdss4,,,[ 3556.3142 3557.1323 3557.9507 ... 10334.747 10337.131 10339.515 ],[ 7.352261 1.6687182 -4.010631 ... 0.3120386 0.43989524\n -3.0100937 ],[0.06552771 0. 0.09258213 ... 0.11384672 0.09031007 0.09989855],5.3429966
290678477446232960,26.33241871174532,23.2978399020685,5112,55895,490,sdss4,,,[ 3556.3142 3557.1323 3557.9507 ... 10337.131 10339.515 10341.888 ],[67.34163 68.68939 70.03662 ... 3.5459442 13.617354 13.534983 ],[0.03259968 0. 0.03401059 ... 0.05315509 0.0391328 0.05458795],41.806816
361278458368372992,12.58871837524935,32.97694024393411,6872,56540,698,sdss4,,,[ 3611.6023 3612.435 3613.266 ... 10356.193 10358.569 10360.958 ],[-4.800425 9.012619 -6.520062 ... 4.385841 2.234324 1.5779594],[0.03948437 0.04028739 0.04367167 ... 0.05019667 0.05344825 0.06314062],4.6341696
520790043346964608,35.98571063849756,68.8184182500658,103683,59952,27021597876080856,sdss5,spSpec-103683-59952-27021597876080856.fits,sdss5/wd_fits/spSpec-103683-59952-27021597876080856.fits,[ 3566.9744 3567.795 3568.6177 ... 10394.412 10396.809 10399.206 ],[-3.8505052e+07 2.1099892e+07 -3.8321398e+06 ... 6.0145645e+01\n 6.0145645e+01 6.0145645e+01],[0.00031186 0.00029126 0.00030737 ... 0. 0. 0. ],25.22791
583949251122623232,137.2107367613618,7.019580223068183,1194,52703,304,sdss4,--,,[3821.2024 3822.0815 3822.963 ... 9181.21 9183.327 9185.439 ],[ 9.318694 15.934431 13.713434 ... 4.6989827 3.3885863 0.8189037],[0.05338169 0.04820895 0.0511077 ... 0.552939 0.59089905 0.60052353],8.560286
590479903515064832,137.62261381911406,9.034338281657385,1300,52973,610,sdss4,--,,[3821.2024 3822.0815 3822.963 ... 9204.495 9206.613 9208.736 ],[100.73954 104.54356 108.79413 ... 9.624027 6.3042235\n 8.438726 ],[0.06008736 0.05637398 0.05510902 ... 0.70071316 0.6419351 0.5354183 ],34.954872
590880916021900672,138.90414180334596,9.726287587275896,1739,53050,12,sdss4,--,,[3805.3972 3806.2727 3807.1504 ... 9200.257 9202.379 9204.495 ],[74.004135 73.47 63.843445 ... 9.485626 9.429831 8.825461],[0.02703905 0.02700883 0.02923179 ... 0.6978134 0.7066595 0.69672817],26.091469
591729395402954880,138.1898762950489,10.41257020188842,102041,59911,27021598149984298,sdss5,spSpec-102041-59911-27021598149984298.fits,sdss5/wd_fits/spSpec-102041-59911-27021598149984298.fits,[ 3566.9744 3567.795 3568.6177 ... 10394.412 10396.809 10399.206 ],[38.804817 89.03215 81.48478 ... 24.822962 24.822962 24.822962],[0.00168974 0.00169494 0.0018988 ... 0. 0. 0. ],14.432425
593650826329812992,138.41747981636797,11.68673313530737,2576,54086,29,sdss4,--,,[3820.3215 3821.2024 3822.0815 ... 9206.613 9208.736 9210.8545],[ 5.4658227 8.948465 13.682929 ... 0.57123053 1.8489103\n -0.28578448],[0.10096005 0.09615636 0.09012982 ... 1.0067292 0.93938506 0.8717232 ],6.4509935


In [35]:
def pseudonormalize(wl, fl, ivar = None, avg_size = 150, ret_cont = False):
    
    fl_norm = np.zeros(np.size(fl))
    fl_cont = np.zeros(np.size(fl))
    
    ivar_yes = 0
    if ivar is not None:
        ivar_yes = 1
        ivar_norm = np.zeros(np.size(fl))
        
    for i in range(np.size(wl)):
        wl_clip = ((wl[i]-avg_size/2)<wl) * (wl<(wl[i]+avg_size/2))
        fl_cont[i] = np.median(fl[wl_clip])
        if ivar_yes:
            ivar_norm[i] = ivar[i]*np.median(fl[wl_clip])**2
    
    fl_norm = fl/fl_cont
    
    if ret_cont:
        if ivar_yes:
            return fl_norm
        else:
            return fl_norm
    else:
        if ivar_yes:
            return fl_norm, ivar_norm
        else:
            return fl_norm

In [36]:
len(catalog)

409

In [ ]:
id_ = []

view = True

balmer_lines = [6562.79, 4861.35, 4340.47, 4101.73, 3970.07, 3889.06, 3835.40]

for i in tqdm(range(len(catalog))):
    id_.append(i)
        
    if view:
        plt.figure(figsize=(12,12))
        
        #plt.subplot(2,2,1)
        plt.plot(catalog['wd_wl'][i], catalog['wd_fl'][i])
        
        min_, max_ = plt.ylim()
        if max_ > 1000:
            mask = (7000 < catalog['wd_wl'][i])
            ymax = max(catalog['wd_fl'][i][mask])
            
            plt.ylim(-0.5*ymax, 4*ymax)
        min_, max_ = plt.ylim()
        
        plt.vlines(balmer_lines, min_, max_, colors = 'black', linestyle = 'dashed', alpha = 0.5)
        plt.grid()
        plt.ylabel(r'Flux [$10^{-17}$ ergs/s/cm2/A]')
        plt.xlabel(r'Wavelength [A]')
        plt.title('White Dwarf Spectrum {}'.format(i))
                
        
        
  
        plt.savefig('spectra/sdss4/wd{}.png'.format(i))

    
catalog['wd_catalog_id'] = id_

  5%|▍         | 20/409 [00:03<01:01,  6.32it/s]<ipython-input-47-698dff42d605>:11: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=(12,12))
  5%|▌         | 21/409 [00:03<01:00,  6.44it/s]<ipython-input-47-698dff42d605>:11: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=(12,12))
  5%|▌         | 22/409 [00:03<00:57,  6.68it/s]<ipython-input-47-698dff42d605>:11: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until ex

In [ ]:
not_da = [0,1]
print(len(not_da))
reject = catalog[not_da]

catalog.remove_rows(not_da)

In [ ]:
from astropy.table import Table, Column, MaskedColumn, join
from astropy.io import fits

nug = fits.open('data/nugget.fits')
teff_prior = True

#print(Table(nug[1].data))

#try:
nug = fits.open('data/nugget.fits')
nug = Table(nug[1].data)

print(nug)

nug.remove_rows(not_da)

#print(catalog['wd_source_id'])
print(nug)
catalog = join(catalog, nug, keys='wd_source_id', join_type='inner')
print('worked')
#except:
#    teff_prior = False

In [ ]:
from lmfit import Parameters, fit_report, minimize

calculate_rvs = True


if calculate_rvs:
    rvs = []
    e_rvs = []
    
    catalog['rv_id'] = np.ones(len(catalog))

    teff = []
    logg = []
    e_teff = []
    e_logg = []

    redchis = []
    drops = []
    figs = []

    return_obj = []

    #catalog['wd_ivar'] = [catalog['wd_ivar'][i]  for i in range(len(catalog))]

    for i in tqdm(range(len(catalog))):
        #print(catalog['phot_teff'][i])
        corvmodel = corv.models.make_koester_model(names = ['a','b','g','d'])

        fl = np.array(catalog['wd_fl'][i])

        #for j in range(len(fl)):
        #    if fl[j] < 0:
        #        fl[j] = 0

        rv, e_rv, redchi, param_res, f = corv.fit.fit_corv(np.array(catalog['wd_wl'][i]), fl, np.array(catalog['wd_ivar'][i]), corvmodel, xcorr_kw = dict(xcorr_kw = dict(plot = True)))

        f.suptitle('%d : %2.2f +/- %2.2f' % (catalog['wd_catalog_id'][i], rv, e_rv))
        f.savefig('spectra/corvfits/fits/{}.png'.format(catalog['wd_catalog_id'][i]))
        
        if param_res.params['teff'].stderr == None:
            param_res.params['teff'].stderr = 1e-6
        if param_res.params['logg'].stderr == None:
            param_res.params['logg'].stderr = 1e-6
        if param_res.params['RV'].stderr == None:
            param_res.params['RV'].stderr = 1e-6

        f = corv.utils.lineplot(np.array(catalog['wd_wl'][i]), np.array(catalog['wd_fl'][i]), np.array(catalog['wd_ivar'][i]), corvmodel, param_res.params, gap = 0.3, printparams = True,
                 figsize = (6, 5))

        f.suptitle('%d : %2.2f +/- %2.2f' % (catalog['wd_catalog_id'][i], rv, e_rv))
        figs.append(f)

        #f.savefig('spectra/corvfits/corvfit{}.jpg'.format(i))

        teff.append(param_res.params['teff'].value)
        logg.append(param_res.params['logg'].value)
        e_teff.append(param_res.params['teff'].stderr)
        e_logg.append(param_res.params['logg'].stderr)

        rvs.append(rv)
        e_rvs.append(e_rv)
        redchis.append(redchi)

        return_obj.append(param_res)
        
        catalog['rv_id'][i] = i



    #catalog.remove_rows(drops)
    catalog['wd_rv'] = rvs
    catalog['wd_e_rv'] = e_rvs

    catalog['wd_corv_logg'] = logg
    catalog['wd_corv_teff'] = teff
    catalog['wd_e_teff'] = e_teff
    catalog['wd_e_logg'] = e_logg
    
    catalog['wd_e_rv'][catalog['wd_e_rv'] == None] = 1e-6
    catalog['wd_rechi'] = redchis
    catalog['figs'] = figs
    
else:
    trv_cat = fits.open('data/wd_rvs.fits')
    rv_cat = Table(trv_cat[1].data)
    
    catalog['wd_rv'] = rv_cat['wd_rv']
    catalog['wd_e_rv'] = rv_cat['wd_e_rv']
    
    catalog['wd_corv_logg'] = rv_cat['wd_corv_logg']
    catalog['wd_corv_teff'] = rv_cat['wd_corv_teff']
    catalog['wd_e_teff'] = rv_cat['wd_e_teff']
    catalog['wd_e_logg'] = rv_cat['wd_e_logg']
    
    catalog['wd_rechi'] = rv_cat['wd_rechi']
    

In [ ]:
if calculate_rvs:
    catalog[['wd_rv', 'wd_e_rv', 'wd_corv_logg', 'wd_corv_teff', 'wd_e_teff', 'wd_e_logg', 'wd_rechi', 'wd_catalog_id']].write('data/wd_rvs.fits', overwrite=True)

In [ ]:
if calculate_rvs:
    for i in range(len(catalog)):
        catalog['figs'][i].savefig('spectra/corvfits/corvfit{}.jpg'.format(catalog['wd_catalog_id'][i]))
        
    catalog.remove_columns(['figs'])

In [ ]:
plt.figure(figsize=(10,10))
plt.grid()
plt.hist(catalog['wd_rv'], bins = 20, histtype='step', color='black')
ymin, ymax = plt.ylim()
plt.vlines(np.mean(catalog['wd_rv']), ymin, ymax, linestyles='dashed')
plt.xlabel(r'$RV_{WD}$', fontsize=18)

In [ ]:
catalog.remove_columns(['wd_wl', 'wd_fl', 'wd_ivar'])
catalog.write('data/03_wd_rv.fits', format='fits', overwrite=True)